In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [38]:
ny = pd.read_table("Data/NY_Temp.txt",sep=',',index_col="DATE")

In [39]:
ny.index.dtype

dtype('int64')

In [40]:
ny.head()

,TAVG
DATE,
1870,53.8
1871,51.3
1872,51.3
1873,50.9
1874,51.3


In [41]:
ny.index = pd.to_datetime(ny.index)

In [42]:
ny.index.dtype

dtype('<M8[ns]')

In [43]:
ny.head()

,TAVG
DATE,
1970-01-01 00:00:00.000001870,53.8
1970-01-01 00:00:00.000001871,51.3
1970-01-01 00:00:00.000001872,51.3
1970-01-01 00:00:00.000001873,50.9
1970-01-01 00:00:00.000001874,51.3


In [48]:
ny.TAVG.tail()

DATE
1970-01-01 00:00:00.000002012    57.3
1970-01-01 00:00:00.000002013    55.3
1970-01-01 00:00:00.000002014    54.4
1970-01-01 00:00:00.000002015    56.7
1970-01-01 00:00:00.000002016    57.2
Name: TAVG, dtype: float64

In [49]:
ny.TAVG.pct_change().tail()

DATE
1970-01-01 00:00:00.000002012    0.015957
1970-01-01 00:00:00.000002013   -0.034904
1970-01-01 00:00:00.000002014   -0.016275
1970-01-01 00:00:00.000002015    0.042279
1970-01-01 00:00:00.000002016    0.008818
Name: TAVG, dtype: float64

In [50]:
ny.TAVG.autocorr()

0.5705478431596489

In [26]:
ny.TAVG.pct_change()

DATE
1970-01-01 00:00:00.000001870         NaN
1970-01-01 00:00:00.000001871   -0.046468
1970-01-01 00:00:00.000001872    0.000000
1970-01-01 00:00:00.000001873   -0.007797
1970-01-01 00:00:00.000001874    0.007859
                                   ...   
1970-01-01 00:00:00.000002012    0.015957
1970-01-01 00:00:00.000002013   -0.034904
1970-01-01 00:00:00.000002014   -0.016275
1970-01-01 00:00:00.000002015    0.042279
1970-01-01 00:00:00.000002016    0.008818
Name: TAVG, Length: 147, dtype: float64

In [62]:
x=[3,5,9,20,12,17,22,23,51,41,56,75,60,75,88]

In [64]:
series=pd.DataFrame(data=y,index=np.arange(1,len(x)+1),columns=['x'])

In [68]:
series['y']=series.shift(1)

In [72]:
series.dropna(inplace=True)

In [76]:
series

,x,y
2,5,3.0
3,9,5.0
4,20,9.0
5,12,20.0
6,17,12.0
7,22,17.0
8,23,22.0
9,51,23.0
10,41,51.0
11,56,41.0


In [78]:
series['x'].corr(series['y'])

0.9045073963463822

In [82]:
new=series.pct_change().dropna()

In [84]:
new.x.corr(new.y)

-0.2686023837749091